# Financial Econometrics I

#### Linear AR and MA models.

by Lukas Vacha and Josef Kurka

#### Seminar 3: Summer Semester 2023/2024
___

The data for seminars can be downloaded on [Google drive](https://drive.google.com/drive/u/0/folders/1_C5kBw9KY59K1J-uRud3G-8kJHfZy_zC)

For this seminar you will need 'data_seminar3.csv' and 'spread.csv'.
___

### ARMA processes 'ingredients'

AR(1)

$$y_t = \phi_0 + \phi_1 y_{t-1} + \epsilon_t$$

MA(1)

$$y_t = \theta_0 + \theta_1 \epsilon_{t-1} + \epsilon_t$$

ARMA(1, 1)

$$y_t = \phi_0 + \phi_1 y_{t-1} + \theta_1 \epsilon_{t-1} + \epsilon_t$$

In [ ]:
if (!require(forecast)) install.packages('forecast')

library(stats)
library(repr)
library(tseries)
library(readr)
library(forecast)

options(repr.plot.width = 12, repr.plot.height = 9)

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 8)
set.seed(229)
par(mfrow = c(1, 2))
eps <- rnorm(100)
phi <- 0.6
phi2 <- 0.3
s1 <- vector()
s2 <- vector()
s1[1] <- 0
s2[1] <- 0
s1[2] <- 0
s2[2] <- 0
for (i in 3:100){
    s1[i] <- phi * s1[i - 1]  + phi2 * s1[i - 2] + eps[i]
    s2[i] <- phi * eps[i - 1] + phi2 * eps[i - 2]+ eps[i]
}
plot.ts(s2, ylab = NA)
plot.ts(s1, ylab = NA)

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 12)
par(mfrow = c(2, 2))
acf(s1)
pacf(s1)
acf(s2)
pacf(s2)

## ARMA models identification and estimation

We are going to work with a couple of particular time series from the ARMA family, and try to identify the "right" order of AR and MA.

We will use the Box-Jenkins method

* Identification
* Estimation
* Model diagnostics

#### Identification

One of the first steps is to look at ACF, and PACF. There some typical ACFs that signal different processes.

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 8)
par(mfrow = c(1, 2))
x <- seq(1, 30)
plot(0.9 ^ x, type = 'h', ylim = c(-1, 1), main = 'AR', ylab = NA)
plot(sin(x) * 1/(sqrt(x)), type = 'h', ylim = c(-1, 1), main = 'AR', ylab = NA)

In [ ]:
x <- c( 0.8, 0.4,  rep(0.05, 3), rep(-0.05, 23))
plot(x, type = 'h', main = 'MA',  ylim = c(-1, 1), , ylab = NA)

In [ ]:
x <- c(0.6, 0.8, -0.5, 0.6, -0.4, rep(0.05, 25))
plot(x, type = 'h', main = 'ARMA',  ylim = c(-1, 1), ylab = NA)

In [ ]:
plot(rnorm(30, sd = 0.05), ylim = c(-1, 1), main = 'WN', type = 'h', ylab = NA)

In [ ]:
x <- seq(1, 30)
plot(1 - 0.01 * x, ylim = c(-1, 1), main = 'Non-stationary', type = 'h', ylab = NA)

In [ ]:
x <- c(rep(0.05, 3), 0.8, rep(0.05, 3), 0.8, rep(0.05, 3), 0.8, rep(0.05, 3), 0.8, rep(0.05, 3), 0.8)
plot(x, type = 'h', main = 'Seasonal',  ylim = c(-1, 1), ylab = NA)

In [ ]:
par(mfrow = c(3, 2))
par(mfrow = c(1, 2))
x <- seq(1, 30)
plot(0.9 ^ x, type = 'h', ylim = c(-1, 1), main = 'AR', ylab = NA)
plot(sin(x) * 1/(sqrt(x)), type = 'h', ylim = c(-1, 1), main = 'AR', ylab = NA)
x <- c(rep(0.05, 2), 0.8, rep(0.05, 3), -0.5, rep(-0.05, 23))
plot(x, type = 'h', main = 'MA',  ylim = c(-1, 1), ylab = NA)
x <- c(0.6, 0.8, -0.5, 0.6, -0.4, rep(0.05, 25))
plot(x, type = 'h', main = 'ARMA',  ylim = c(-1, 1), ylab = NA)
plot(rnorm(30, sd = 0.05), ylim = c(-1, 1), main = 'WN', type = 'h', ylab = NA)
x <- seq(1, 30)
plot(1 - 0.01 * x, ylim = c(-1, 1), main = 'Non-stationary', type = 'h', ylab = NA)

Load the data and observe the Box-Jenkins Method in practice.

In [ ]:
proc <- read.csv("data_seminar3.csv")
# Data from Google Drive may be comma separated
spr <- read.csv("spread.csv", sep = ",")

spr$Spread <- as.numeric(as.character(spr$Spread))
colnames(spr) <- c('Year', 'Quarter', 'Value')
spr$Year <- as.Date(paste(spr$Year, paste(0, spr$Quarter, sep = ""), 01,  sep = "-"), format = "%Y-%m-%d")
spr <- spr[, -2]

In [ ]:
head(proc)
head(spr)

y1 <- proc$Y1
y2 <- proc$Y2
y3 <- proc$Y3

spread <- spr$Value

Y1, Y2, and Y3 are simulated realizations of particular processes from ARMA family. Our task is to try find out what is the underlying process in each case. The answers can be found on the bottom of each exercise.

### Y1

Let's start with series Y1. Begin with plotting the data.

In [ ]:
library(repr)
#install.packages('tseries')
library(tseries)
plot.ts(y1, ylab = NA, main = 'Y1')


We can't say much based on the plot. What are the properties of this time series? Is it stationary?
How about autocorrelations?


In [ ]:
par(mfrow = c(1, 2))
acf(y1)
pacf(y1)

We would not rule out stationarity based on ACF and PACF. Conduct a formal test for stationarity.

In [ ]:
adf.test(y1, k = 1)

We would reject non-stationarity at 10% level. If we believe time series is stationary, we can the start the process of estimating the order of ARMA(p, q) process.

Pacf shows a very significant dependence at first lag, therefore we start estimation with AR(1) process.

In [ ]:
y1_ar1 <- Arima(y1, order = c(1, 0, 0))
summary(y1_ar1)

This summary itself is not very informative. We can see, that the intercept, although large in magnitude has a huge standard error. Examine ACF and PACF of the residuals. If the model we chose is a good fit, there should not be much dependence left.

In [ ]:
par(mfrow = c(1, 2))
acf(y1_ar1$residuals, main = NA)
pacf(y1_ar1$residuals, main = NA)

In addition to the plots, we should test for joint siginificance of residuals' autocorrelation. As we already know, this can be done using the Ljung-Box Q test. Let's test for joint significance up to 4th, 8th and 12th lag respectively.

In [ ]:
Box.test(y1_ar1$residuals, type = "Ljung-Box", lag = 4)
Box.test(y1_ar1$residuals, type = "Ljung-Box", lag = 8)
Box.test(y1_ar1$residuals, type = "Ljung-Box", lag = 12)

Recall that the null hypothesis is joint insignificance of autocorrelation coefficients 1 to m. We reject joint insignificance for neither 4, 8 or 12 lags.

Next, try if ARMA(1, 1) would not be a better fit.

In [ ]:
y1_arma11 <- Arima(y1, order = c(1, 0, 1))
summary(y1_arma11)

par(mfrow = c(1, 2))
acf(y1_arma11$residuals, main = NA)
pacf(y1_arma11$residuals, main = NA)

In [ ]:
Box.test(y1_arma11$residuals, type = "Ljung-Box", lag = 4)
Box.test(y1_arma11$residuals, type = "Ljung-Box", lag = 8)
Box.test(y1_arma11$residuals, type = "Ljung-Box", lag = 12)

The dependence in residuals seems to be removed in this case either. When we need to decide which of the multiple suitable models is the "best" for our data, we can compare the information criteria, and choose the model where they are minimalized. Arima function directly reports Bayesian Information Criterion (BIC), Akaike Information Criterion (AIC), and AIC corrected for small samples (AICc). Let's compare AIC and BIC of the models we estimated.

In [ ]:
models <- 2
criteria <- matrix(ncol = 2, nrow = models)
colnames(criteria) <- c('AIC', 'BIC')
rownames(criteria) <- c('AR(1)', 'ARMA(1,1)')

criteria[1, 1] <- y1_ar1$aic
criteria[1, 2] <- y1_ar1$bic
criteria[2, 1] <- y1_arma11$aic
criteria[2, 2] <- y1_arma11$bic

criteria

Both criteria are lower for the AR(1) model. The data are in fact a realization of following AR(1) process

$$ y_t = 0.7 y_{t-1} + \epsilon_t.$$

#### Exercise:

Try applying the Box-Jenkins on the Y2 series.

Hint:

1) Identification - look at the plot of the series, its ACF and PACF. Based on the ACF try to get an idea which could be the underlying process. Test for stationarity.

2) Estimation - estimate the parameters of your candidate processes.

3) Diagnostics - Check the dependencies in the residuals. If you have more suitable candidate models, use the information criteria. If you still have more suitable models, prefer parsimony.

### Y2

In [ ]:
plot.ts(y2, ylab = NA, main = 'Y2')

par(mfrow = c(1, 2))
acf(y2)
pacf(y2)

In [ ]:
y2_ma2 <- Arima(y2, order = c(0, 0, 2))
summary(y2_ma2)

par(mfrow = c(1, 2))
acf(y2_ma2$residuals, main = NA)
pacf(y2_ma2$residuals, main = NA)
Box.test(y2_ma2$residuals, type = "Ljung-Box", lag = 4)
Box.test(y2_ma2$residuals, type = "Ljung-Box", lag = 8)

In [ ]:
y2_ma8 <- Arima(y2, order = c(0, 0, 8))
summary(y2_ma8)

par(mfrow = c(1, 2))
acf(y2_ma8$residuals, main = NA)
pacf(y2_ma8$residuals, main = NA)
Box.test(y2_ma8$residuals, type = "Ljung-Box", lag = 4)
Box.test(y2_ma8$residuals, type = "Ljung-Box", lag = 8)

In [ ]:
y2_ar2 <- Arima(y2, order = c(2, 0, 0))
summary(y2_ar2)

par(mfrow = c(1, 2))
acf(y2_ar2$residuals, main = NA)
pacf(y2_ar2$residuals, main = NA)
Box.test(y2_ar2$residuals, type = "Ljung-Box", lag = 4)
Box.test(y2_ar2$residuals, type = "Ljung-Box", lag = 8)

In [ ]:
y2_arma2 <- Arima(y2, order = c(1, 0, 1))
summary(y2_arma2)

par(mfrow = c(1, 2))
acf(y2_arma2$residuals, main = NA)
pacf(y2_arma2$residuals, main = NA)
Box.test(y2_arma2$residuals, type = "Ljung-Box", lag = 4)
Box.test(y2_arma2$residuals, type = "Ljung-Box", lag = 8)

In [ ]:
models <- 3
criteria <- matrix(ncol = 2, nrow = models)
colnames(criteria) <- c('AIC', 'BIC')
rownames(criteria) <- c('AR(2)', 'ARMA(1,1)', 'MA(2)')

criteria[1, 1] <- y2_ar2$aic
criteria[1, 2] <- y2_ar2$bic
criteria[2, 1] <- y2_arma2$aic
criteria[2, 2] <- y2_arma2$bic
criteria[3, 1] <- y2_ma2$aic
criteria[3, 2] <- y2_ma2$bic

criteria

Both criteria are minimized for ARMA(1, 1), which Y2 is a realization of. The underlying process is

$$ y_t = - 0.7 y_{t-1} - 0.7 \epsilon_{t-1} + \epsilon_t $$

### Y3

In [ ]:
plot.ts(y3, ylab = NA, main = 'Y3')

par(mfrow = c(1, 2))
acf(y3)
pacf(y3)

Especially the first two lags of PACF are significant, but there are also lags of very high order that exceed the confidence band. We will estimate AR(2), and see what happens.

In [ ]:
y3_ar2 <- Arima(y3, order = c(2, 0, 0))
summary (y3_ar2)

par(mfrow = c(1, 2))
acf(y3_ar2$residuals, main = NA)
pacf(y3_ar2$residuals, main = NA)

In [ ]:
Box.test(y3_ar2$residuals, type = "Ljung-Box", lag = 1)
Box.test(y3_ar2$residuals, type = "Ljung-Box", lag = 8)
Box.test(y3_ar2$residuals, type = "Ljung-Box", lag = 12)

According to the plot, there could be some week dependence at high lags of PACF. We could estimate AR(17), but that hardly makes sense.

In [ ]:
y3_ar17 <- Arima(y3, order = c(0, 0, 2))
summary (y3_ar17)

par(mfrow = c(1, 2))
acf(y3_ar17$residuals, main = NA)
pacf(y3_ar17$residuals, main = NA)
Box.test(y3_ar17$residuals, type = "Ljung-Box", lag = 1)
Box.test(y3_ar17$residuals, type = "Ljung-Box", lag = 8)

In [ ]:
plot.ts(y3, ylab = NA, main = 'AR(2) fitted values')
lines(y3_ar2$fitted, col = 'red')

plot.ts(y3, ylab = NA, main = 'AR(17) fitted values')
lines(y3_ar17$fitted, col = 'red')

With AR(17) the ACF and PACF look better (possibly due to overfitting), however the model is hardly much better in terms of fit. Moreover, one should prefer model simplicity.

It could be the case that different parts of the sample can be characterized by different models. Subsample the data to two halves.

In [ ]:
y3_01 <- y3[1:(length(y3) / 2)]
y3_02 <- y3[((length(y3) / 2) + 1):length(y3)]

options(repr.plot.width = 10, repr.plot.height = 10)
par(mfrow = c(2, 2))

acf(y3_01, main = "First subsample")
pacf(y3_01, main = "First subsample")
acf(y3_02, main = "Second subsample")
pacf(y3_02, main = "Second subsample")

In [ ]:
model_sub1 <- Arima(y3_01, order = c(2, 0, 0))
model_sub2 <- Arima(y3_02, order = c(2, 0, 0))

summary(model_sub1)
summary(model_sub2)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
par(mfrow = c(2, 2))

acf(model_sub1$residuals, main = "First subsample", lag = 20)
pacf(model_sub1$residuals, main = "First subsample", lag = 20)
acf(model_sub2$residuals, main = "Second subsample", lag = 20)
pacf(model_sub2$residuals, main = "Second subsample", lag = 20)

In [ ]:
Box.test(model_sub1$residuals, type = "Ljung-Box", lag = 4)
Box.test(model_sub1$residuals, type = "Ljung-Box", lag = 8)
Box.test(model_sub1$residuals, type = "Ljung-Box", lag = 12)

Box.test(model_sub2$residuals, type = "Ljung-Box", lag = 4)
Box.test(model_sub2$residuals, type = "Ljung-Box", lag = 8)
Box.test(model_sub2$residuals, type = "Ljung-Box", lag = 12)

When we allow the coefficients to be different in first and second part of the sample, the dependence in residuals disappears. Note that we still estimate AR(2) process. Y3 is a realization of  AR(2) process specified as follows:

$$ y_t = 0.7 y_{t-1} - 0.49 y_{t-2} + \epsilon_t.$$

R has a built-in function for estimating the order of ARIMA processes. It goes through the set of possible lag (and differencing) combinations, and chooses the best one based on information criteria. One must be careful while using it, as the criteria might favor overfitted models. Successive model diagnostics is an essential part of estimation.

See what happens if we let auto.arima select the model for the three series we just examined.

In [ ]:
auto.arima(y2)

For y2, the function estimates the same model as we did.

In [ ]:
auto.arima(y3, approximation = FALSE, stepwise = FALSE)

For y3, the machine prefers ARMA(2, 3), but look at the first AR coefficient. Let's compare all three of the criteria between AR(2) and ARMA(2, 3).

In [ ]:
model <- Arima(y3, order = c(2, 0, 0), include.mean = FALSE)

models <- 2
criteria <- matrix(ncol = 3, nrow = models)
colnames(criteria) <- c('AIC', 'AICc', 'BIC')
rownames(criteria) <- c('AR(2)','ARMA(2, 3)')

criteria[1, 1] <- model$aic
criteria[1, 2] <- model$aicc
criteria[1, 3] <- model$bic
criteria[2, 1] <- auto.arima(y3)$aic
criteria[2, 2] <- auto.arima(y3)$aicc
criteria[2, 3] <- auto.arima(y3)$bic

criteria

The criteria do not send a clear message, but AR(2) may be more preferable due to its parsimony.

In [ ]:
auto.arima(y1)

Here, R estimates the model with first difference. We get to drawbacks of possible over-differencing later. For now, let's tell R not to do that.

In [ ]:
auto.arima(y1, max.d = 0)

#### Spread

Now we will work with real-world data, thus we don't know what process the data come from. Start again by plotting the time series.

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 6)

plot.ts(spr$Value, xlab = 'Year', ylab = NA, main = 'SPREAD', axes = F)
axis(1, at = c(1, 41, 81, 121, 161), lab = c('1960', '1970', '1980', '1990', '2000'))
box()

par(mfrow = c(1, 2))
acf(spread, main = NA, lag = 50)
pacf(spread, main = NA, lag = 50)

Test for stationarity.

In [ ]:
adf.test(spread, k = 1)
adf.test(spread, k = 5)
adf.test(spread, k = 10)

We reject non-stationarity based on ADF test. There seems to be a lot of dependence visible from the ACF and PACF.

In [ ]:
auto <- auto.arima(spread, max.d = 0)
auto

par(mfrow = c(1, 2))
acf(auto$residuals, main = NA)
pacf(auto$residuals, main = NA)

#### AR(7)

In [ ]:
fit1 <- Arima(spread, order = c(7, 0, 0))
summary(fit1)

par(mfrow = c(1, 2))|
acf(fit1$residuals, main = NA)
pacf(fit1$residuals, main = NA)

In [ ]:
Box.test(fit1$residuals, type = "Ljung-Box", lag = 1)
Box.test(fit1$residuals, type = "Ljung-Box", lag = 8)
Box.test(fit1$residuals, type = "Ljung-Box", lag = 12)

#### AR(2)

In [ ]:
fit2 <- Arima(spread, order = c(2, 0, 0))
summary(fit2)

par(mfrow = c(1, 2))
acf(fit2$residuals, main = NA)
pacf(fit2$residuals, main = NA)

In [ ]:
Box.test(fit2$residuals, type = "Ljung-Box", lag = 4)
Box.test(fit2$residuals, type = "Ljung-Box", lag = 8)
Box.test(fit2$residuals, type = "Ljung-Box", lag = 12)

#### AR(1; 2; 7)

To eliminate the "mirror" coefficients from AR(7) model, we fit the AR(7) again, but estimate only 1st, 2nd, and 7th coefficient, while the others are set to 0.

Note that fixing some paramaters of ARMA(p, q) to zero (or other value) while fitting a model using *Arima* is done using the *fixed* argument. The order of parameters is $(\phi_1, ..., \phi_p, \psi_1, ..., \psi_q, intercept)$, where $\phi_i$ denotes i-th autoregressive parameter and $\psi_j$ denotes j-th moving average parameter. To make R estimate the parameter enter *NA*. Also, you should have sufficient evidence from the stepwise model building, that restricting certain parameter to a particular value makes sense.

In [ ]:
fit3 <- Arima(spread, order = c(7, 0, 0), fixed = c(NA, NA, 0, 0, 0, 0, NA, NA))
summary(fit3)

par(mfrow = c(1, 2))
acf(fit3$residuals, main = NA)
pacf(fit3$residuals, main = NA)

In [ ]:
Box.test(fit3$residuals, type = "Ljung-Box", lag = 4)
Box.test(fit3$residuals, type = "Ljung-Box", lag = 8)
Box.test(fit3$residuals, type = "Ljung-Box", lag = 12)

#### ARMA(1, 1)

In [ ]:
fit4 <- Arima(spread, order = c(1, 0, 1))
summary(fit4)

par(mfrow = c(1, 2))
acf(fit4$residuals, main = NA)
pacf(fit4$residuals, main = NA)

In [ ]:
Box.test(fit4$residuals, type = "Ljung-Box", lag = 4)
Box.test(fit4$residuals, type = "Ljung-Box", lag = 8)
Box.test(fit4$residuals, type = "Ljung-Box", lag = 12)

#### ARMA(2, 1)

In [ ]:
fit5 <- Arima(spread, order = c(2, 0, 1))
summary(fit5)

par(mfrow = c(1, 2))
acf(fit5$residuals, main = NA)
pacf(fit5$residuals, main = NA)

In [ ]:
Box.test(fit5$residuals, type = "Ljung-Box", lag = 4)
Box.test(fit5$residuals, type = "Ljung-Box", lag = 8)
Box.test(fit5$residuals, type = "Ljung-Box", lag = 12)

#### ARMA(2, {1; 7})

In [ ]:
fit6 <- Arima(spread, order = c(2, 0, 7), fixed = c(NA, NA, NA, 0, 0, 0, 0, 0, NA, NA))
summary(fit6)

par(mfrow = c(1, 2))
acf(fit6$residuals, main = NA)
pacf(fit6$residuals, main = NA)

In [ ]:
Box.test(fit6$residuals, type = "Ljung-Box", lag = 4)
Box.test(fit6$residuals, type = "Ljung-Box", lag = 8)
Box.test(fit6$residuals, type = "Ljung-Box", lag = 12)

There is a lot of possible models, let's see the information criteria.

In [ ]:
models <- 7
criteria <- matrix(ncol = 2, nrow = models)
colnames(criteria) <- c('AIC', 'BIC')
rownames(criteria) <- c('AR(7)', 'AR(2)', 'AR(1; 2; 7)', 'ARMA(1, 1)', 'ARMA(2, 1)', 'ARMA(2, {1; 7})', 'automatic - ARMA(1, 3)')

criteria[1, 1] <- fit1$aic
criteria[1, 2] <- fit1$bic
criteria[2, 1] <- fit2$aic
criteria[2, 2] <- fit2$bic
criteria[3, 1] <- fit3$aic
criteria[3, 2] <- fit3$bic
criteria[4, 1] <- fit4$aic
criteria[4, 2] <- fit4$bic
criteria[5, 1] <- fit5$aic
criteria[5, 2] <- fit5$bic
criteria[6, 1] <- fit6$aic
criteria[6, 2] <- fit6$bic
criteria[7, 1] <- auto$aic
criteria[7, 2] <- auto$bic

criteria

#### Subsetting?

The results do not provide a clear message, we might want to select ARMA(2, 1), but let's look at the data once again first.

In [ ]:
plot.ts(spr$Value, xlab = 'Year', ylab = NA, main = 'SPREAD', axes = F, cex.main = 0.8)
axis(1, at = c(1, 41, 81, 121, 161), lab = c('1960', '1970', '1980', '1990', '2000'))
box()

There is a suspicious jump around 1980. What if the process has a shift? Examine the data between 1980 and 1983

In [ ]:
spr[spr[, 1] > as.Date("1980-01-01") & spr[, 1] < as.Date("1984-01-01"),]

The jump occurs between 3rd and 4th quarter of 1981. Let's subsample the data.

In [ ]:
spread1 <- spr[spr[, 1] <= as.Date("1981-03-01"), 'Value']
spread2 <- spr[spr[, 1] > as.Date("1981-03-01"), 'Value']

par(mfrow = c(1, 2))
plot.ts(spread1, ylab = NA, main = 'Until 1982', ylim = c(-2, 4))
plot.ts(spread2, ylab = NA, main = 'Since 1982', ylim = c(-2, 4))

Examine ACF and PACF of both subsamples.

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)

par(mfrow = c(2, 2))

acf(spread1, main = 'Until 1982', lag = 50)
pacf(spread1, main = 'Until 1982', lag = 50)
acf(spread2, main = 'Since 1982', lag = 50)
pacf(spread2, main = 'Since 1982', lag = 50)

There are some differences in the correlation functions. Fit the same models as we did for the whole sample for both subsamples individually.

In [ ]:
sub1_fit1 <- Arima(spread1, order = c(7, 0, 0))
sub2_fit1 <- Arima(spread2, order = c(7, 0, 0))

summary(sub1_fit1)
summary(sub2_fit1)

par(mfrow = c(2, 2))
acf(sub1_fit1$residuals, main = 'Subsample 1')
pacf(sub1_fit1$residuals, main = 'Subsample 1')
acf(sub2_fit1$residuals, main = 'Subsample 2')
pacf(sub2_fit1$residuals, main = 'Subsample 2')

In [ ]:
Box.test(sub1_fit1$residuals, type = "Ljung-Box", lag = 4)
Box.test(sub1_fit1$residuals, type = "Ljung-Box", lag = 8)
Box.test(sub1_fit1$residuals, type = "Ljung-Box", lag = 12)

Box.test(sub2_fit1$residuals, type = "Ljung-Box", lag = 4)
Box.test(sub2_fit1$residuals, type = "Ljung-Box", lag = 8)
Box.test(sub2_fit1$residuals, type = "Ljung-Box", lag = 12)

In [ ]:
sub1_fit2 <- Arima(spread1, order = c(2, 0, 0))
sub2_fit2 <- Arima(spread2, order = c(2, 0, 0))

summary(sub1_fit2)
summary(sub2_fit2)

par(mfrow = c(2, 2))
acf(sub1_fit2$residuals, main = 'Subsample 1')
pacf(sub1_fit2$residuals, main = 'Subsample 1')
acf(sub2_fit2$residuals, main = 'Subsample 2')
pacf(sub2_fit2$residuals, main = 'Subsample 2')

In [ ]:
Box.test(sub1_fit2$residuals, type = "Ljung-Box", lag = 4)
Box.test(sub1_fit2$residuals, type = "Ljung-Box", lag = 8)
Box.test(sub1_fit2$residuals, type = "Ljung-Box", lag = 12)

Box.test(sub2_fit2$residuals, type = "Ljung-Box", lag = 4)
Box.test(sub2_fit2$residuals, type = "Ljung-Box", lag = 8)
Box.test(sub2_fit2$residuals, type = "Ljung-Box", lag = 12)

In [ ]:
sub1_fit3 <- Arima(spread1, order = c(7, 0, 0), fixed = c(NA, NA, 0, 0 ,0, 0, NA, NA))
sub2_fit3 <- Arima(spread2, order = c(7, 0, 0), fixed = c(NA, NA, 0, 0 ,0, 0, NA, NA))

summary(sub1_fit3)
summary(sub2_fit3)

par(mfrow = c(2, 2))
acf(sub1_fit3$residuals, main = 'Subsample 1')
pacf(sub1_fit3$residuals, main = 'Subsample 1')
acf(sub2_fit3$residuals, main = 'Subsample 2')
pacf(sub2_fit3$residuals, main = 'Subsample 2')

In [ ]:
Box.test(sub1_fit3$residuals, type = "Ljung-Box", lag = 4)
Box.test(sub1_fit3$residuals, type = "Ljung-Box", lag = 8)
Box.test(sub1_fit3$residuals, type = "Ljung-Box", lag = 12)

Box.test(sub2_fit3$residuals, type = "Ljung-Box", lag = 4)
Box.test(sub2_fit3$residuals, type = "Ljung-Box", lag = 8)
Box.test(sub2_fit3$residuals, type = "Ljung-Box", lag = 12)

In [ ]:
sub1_fit4 <- Arima(spread1, order = c(1, 0, 1))
sub2_fit4 <- Arima(spread2, order = c(1, 0, 1))

summary(sub1_fit4)
summary(sub2_fit4)

par(mfrow = c(2, 2))
acf(sub1_fit4$residuals, main = 'Subsample 1')
pacf(sub1_fit4$residuals, main = 'Subsample 1')
acf(sub2_fit4$residuals, main = 'Subsample 2')
pacf(sub2_fit4$residuals, main = 'Subsample 2')

In [ ]:
Box.test(sub1_fit4$residuals, type = "Ljung-Box", lag = 4)
Box.test(sub1_fit4$residuals, type = "Ljung-Box", lag = 8)
Box.test(sub1_fit4$residuals, type = "Ljung-Box", lag = 12)

Box.test(sub2_fit4$residuals, type = "Ljung-Box", lag = 4)
Box.test(sub2_fit4$residuals, type = "Ljung-Box", lag = 8)
Box.test(sub2_fit4$residuals, type = "Ljung-Box", lag = 12)

In [ ]:
sub1_fit5 <- Arima(spread1, order = c(2, 0, 1))
sub2_fit5 <- Arima(spread2, order = c(2, 0, 1))

summary(sub1_fit5)
summary(sub2_fit5)

par(mfrow = c(2, 2))
acf(sub1_fit5$residuals, main = 'Subsample 1')
pacf(sub1_fit5$residuals, main = 'Subsample 1')
acf(sub2_fit5$residuals, main = 'Subsample 2')
pacf(sub2_fit5$residuals, main = 'Subsample 2')

In [ ]:
Box.test(sub1_fit5$residuals, type = "Ljung-Box", lag = 4)
Box.test(sub1_fit5$residuals, type = "Ljung-Box", lag = 8)
Box.test(sub1_fit5$residuals, type = "Ljung-Box", lag = 12)

Box.test(sub2_fit5$residuals, type = "Ljung-Box", lag = 4)
Box.test(sub2_fit5$residuals, type = "Ljung-Box", lag = 8)
Box.test(sub2_fit5$residuals, type = "Ljung-Box", lag = 12)

In [ ]:
sub1_fit6 <- Arima(spread1, order = c(2, 0, 7), fixed = c(NA, NA, NA, 0, 0, 0, 0, 0, NA, NA))
sub2_fit6 <- Arima(spread2, order = c(2, 0, 7), fixed = c(NA, NA, NA, 0, 0, 0, 0, 0, NA, NA))

summary(sub1_fit6)
summary(sub2_fit6)

par(mfrow = c(2, 2))
acf(sub1_fit6$residuals, main = 'Subsample 1')
pacf(sub1_fit6$residuals, main = 'Subsample 1')
acf(sub2_fit6$residuals, main = 'Subsample 2')
pacf(sub2_fit6$residuals, main = 'Subsample 2')

In [ ]:
Box.test(sub1_fit6$residuals, type = "Ljung-Box", lag = 4)
Box.test(sub1_fit6$residuals, type = "Ljung-Box", lag = 8)
Box.test(sub1_fit6$residuals, type = "Ljung-Box", lag = 12)

Box.test(sub2_fit6$residuals, type = "Ljung-Box", lag = 4)
Box.test(sub2_fit6$residuals, type = "Ljung-Box", lag = 8)
Box.test(sub2_fit6$residuals, type = "Ljung-Box", lag = 12)

In [ ]:
models <- 6
criteria <- matrix(ncol = 4, nrow = models)
colnames(criteria) <- c('AIC 1', 'BIC 1', 'AIC 2', 'BIC 2')
rownames(criteria) <- c('AR(7)', 'AR(2)', 'AR(1)', 'ARMA(1,1)', 'ARMA(2,1)', 'ARMA(2,7)')

criteria[1, 1] <- sub1_fit1$aic
criteria[1, 2] <- sub1_fit1$bic
criteria[2, 1] <- sub1_fit2$aic
criteria[2, 2] <- sub1_fit2$bic
criteria[3, 1] <- sub1_fit3$aic
criteria[3, 2] <- sub1_fit3$bic
criteria[4, 1] <- sub1_fit4$aic
criteria[4, 2] <- sub1_fit4$bic
criteria[5, 1] <- sub1_fit5$aic
criteria[5, 2] <- sub1_fit5$bic
criteria[6, 1] <- sub1_fit6$aic
criteria[6, 2] <- sub1_fit6$bic

criteria[1, 3] <- sub2_fit1$aic
criteria[1, 4] <- sub2_fit1$bic
criteria[2, 3] <- sub2_fit2$aic
criteria[2, 4] <- sub2_fit2$bic
criteria[3, 3] <- sub2_fit3$aic
criteria[3, 4] <- sub2_fit3$bic
criteria[4, 3] <- sub2_fit4$aic
criteria[4, 4] <- sub2_fit4$bic
criteria[5, 3] <- sub2_fit5$aic
criteria[5, 4] <- sub2_fit5$bic
criteria[6, 3] <- sub2_fit6$aic
criteria[6, 4] <- sub2_fit6$bic

criteria

Different models would be selected for period 1 and period 2.

In [ ]:
auto.arima(spread1)
auto.arima(spread2)
auto.arima(spread2, max.d = 0)

### Caveats of auto.arima function

To illustrate the problem with automatic fitting with the auto.arima function, simulate a realization of AR(1) with coefficient 0.5 under the following seed.

In [ ]:
set.seed(2083)
ar <- arima.sim(n = 500, model = list(ar = c(0.5)))

autofit <- auto.arima(ar)
fit <- Arima(ar, order = c(1, 0, 0))

autofit
fit

par(mfrow = c(1, 2))
acf(autofit$residuals, main = 'ACF - ARMA(2, 3)')
pacf(autofit$residuals, main = 'PACF - ARMA(2, 3)')

acf(fit$residuals, main = 'ACF - AR(1)')
pacf(autofit$residuals, main = 'PACF - AR(1)')

In [ ]:
models <- 2
criteria <- matrix(ncol = 2, nrow = models)
colnames(criteria) <- c('AIC', 'BIC')
rownames(criteria) <- c( 'AR(1)', 'ARMA(2, 3)')

criteria[1, 1] <- fit$aic
criteria[1, 2] <- fit$bic
criteria[2, 1] <- autofit$aic
criteria[2, 2] <- autofit$bic

criteria

par(mfrow = c(2, 1))

plot.ts(ar, ylab = NA, main = 'AR(1)')
lines(fit$fitted, col = 'green')

plot.ts(ar, ylab = NA, main = 'ARMA(2, 3)')
lines(fit$fitted, col = 'red')

On this example we can see, that although the sequence is a realization of AR(1), automatical R estimation fits ARMA(2, 3). However, looking at the criteria, residuals, and fit there is no particular reason to prefer ARMA(2, 3) over AR(1).

### Addition of two processes

#### MA($q_1$) + MA($q_2$)

Simulate realizations of MA(1) and MA(2) processes of length 500.

In [ ]:
set.seed(1450)
ma1 <- arima.sim(n = 500, model = list(ma = 0.6))
ma2 <- arima.sim(n = 500, model = list(ma = c(0.7, -0.5)))

options(repr.plot.width = 10, repr.plot.height = 8)
par(mfrow = c(1, 2))
plot.ts(ma1, ylab = NA, main = 'MA(1)')
plot.ts(ma2, ylab = NA, main = 'MA(2)')

options(repr.plot.width = 10, repr.plot.height = 10)
par(mfrow = c(2, 2))
acf(ma1, main = 'MA(1)')
pacf(ma1, main = 'MA(1)')

acf(ma2, main = 'MA(2)')
pacf(ma2, main = 'MA(2)')

auto.arima(ma1)
auto.arima(ma2)

Now sum these two simulated processes. What will be the properties of the resulting process?

In [ ]:
MA <- ma1 + ma2

options(repr.plot.width = 8, repr.plot.height = 6)
plot.ts(MA, ylab = NA, main = 'MA(1) + MA(2)')

par(mfrow = c(1, 2))
acf(MA)
pacf(MA)

MAfit <- auto.arima(MA)
MAfit

plot.ts(MA)
lines(MAfit$fitted, col = 'red')

par(mfrow = c(1,2))
acf(MAfit$residuals, main = NA)
pacf(MAfit$residuals, main = NA)

If we sum $MA(q_1)$, and $MA(q_2)$ process, the resulting process will be $MA(max \{ q_1, q_2 \})$. In this particular case, we get a very decent fit by the MA(2) from the automatic fitting function.

#### AR(p) + AR(q)

Simulate realizations of two AR(1) processes with coefficients 0.5 and 0.4.

In [ ]:
set.seed(5683)
ar1 <- arima.sim(n = 500, model = list(ar = 0.5))
ar2 <- arima.sim(n = 500, model = list (ar = 0.4))

options(repr.plot.width = 10, repr.plot.height = 8)
par(mfrow = c(1, 2))
plot.ts(ar1, ylab = NA, main = 'First AR(1)')
plot.ts(ar2, ylab = NA, main = 'Second AR(1)')

options(repr.plot.width = 10, repr.plot.height = 10)
par(mfrow = c(2, 2))
acf(ar1, main = 'First AR(1)')
pacf(ar1, main = 'First AR(1)')

acf(ar2, main = 'Second AR(1)')
pacf(ar2, main = 'Second AR(1)')

auto.arima(ar1)
auto.arima(ar2)

Now sum these two processes. What will be the result?

In [ ]:
AR <- ar1 + ar2

options(repr.plot.width = 8, repr.plot.height = 6)
par(mfrow = c(1, 1))
plot.ts(AR, ylab = NA, main = 'AR(1) + AR(1)')

par(mfrow = c(1, 2))
acf(AR)
pacf(AR)

ARfit <- auto.arima(AR)
ARfit

par(mfrow = c(1, 1))
plot.ts(AR)
lines(ARfit$fitted, col = 'red')

par(mfrow = c(1, 2))
acf(ARfit$residuals, main = NA)
pacf(ARfit$residuals, main = NA)

If $AR(p_1)$ nad $AR(p_2)$ processes are summed, the resulting process is $ARMA(p_1 + p_2, max \{ p_1, p_2 \})$.

### Over-differencing

Simulate a realization of AR(2) process with $\phi_1 = 1.5$, $\phi_2 = -0.5$.

In [ ]:
set.seed(555)
y <- arima.sim(n = 500, model = list(ar = c(1.5, -0.5)))
plot.ts(y, ylab = NA)

The pre-built function won't let us do the simulation, as the process wouldn't be stationary. We have to perform the simulation ourselves.

In [ ]:
set.seed(355)
l <- 502
e <- rnorm(l)
y <- vector()
phi1 <- 1.5
phi2 <- -0.5
y[1] <- 0
y[2] <- phi1 * e[1]
for (i in 3 : l){
    y[i] <- phi1 * y[i - 1] + phi2 * y[i - 2] + e[i]
}

plot.ts(y, ylab = NA)

adf.test(y, k = 1)

Non-stationarity is not rejected. Common procedure in such cases is to remove non-stationarity by differencing. Compute first and second difference.

In [ ]:
first_diff <- diff(y)
second_diff <- diff(y, differences = 2)

plot.ts(first_diff)
plot.ts(second_diff)

adf.test(first_diff, k = 1)
adf.test(second_diff, k = 1)

ADF test suggest stationarity of both first and second difference. Let's see how we would estimate each series.

In [ ]:
fit_y <- auto.arima(y)
fit_y

fit_fd <- auto.arima(first_diff)
fit_fd

autofit_sd <- auto.arima(second_diff)
autofit_sd

Is this "correct"? Examine the individual processes.

The **original process** is AR(2) with unit-root

$$y_t = 1.5 y_{t-1} - 0.5 y_{t-2} + \epsilon_t .$$

After **first differencing**

$$y_t - y_{t-1} = \Delta y_t = 1.5 y_{t-1} - 0.5 y_{t-2} + \epsilon_t - y_{t-1} = 0.5 y_{t-1} - 0.5 y_{t-2} + \epsilon_t \\ = 0.5 \Delta y_{t-1} + \epsilon_t.$$

Hence, first differenced series becomes an AR(1) process with $\phi_1 = 0.5$.

Subtract $\Delta y_{t-1}$ to obtain **second difference**.

Hint: After subtracting $\Delta y_{t-1}$, express $\Delta y_{t-1}$ in terms of $\Delta y_{t-2}$ and epsilon using the first differenced formula.

$$ \Delta y_t - \Delta y_{t-1} = \Delta^2 y_t = 0.5 \Delta y_{t-1} + \epsilon_t - \Delta y_{t-1} = 0.5 \Delta y_{t-1} + \epsilon_t - 0.5 \Delta y_{t-2} - \epsilon_{t-1} \\ = 0.5 \Delta^2 y_{t-1} + \epsilon_t - \epsilon_{t-1}.$$

Hence the over-differenced series is an ARMA(1, 1), and has an invertible error-term.

In [ ]:
fit_sd <- Arima(second_diff, order = c(1, 0 , 1), include.mean = FALSE)
fit_sd

options(repr.plot.width = 10, repr.plot.height = 10)
par(mfrow = c(2, 1))
plot.ts(second_diff, main = 'Autofit')
lines(autofit_sd$fitted, col = 'red')
plot.ts(second_diff, main = 'Manual fit')
lines(fit_sd$fitted, col = 'green')

### Common factor problem

Consider a process
$$y_t = 0.4 y_{t -1} + 0.45 y_{t - 2} + \epsilon_t + \epsilon_{t - 1} + 0.25 \epsilon_{t - 2}.$$
In the lag operator form, we have
$$(1 - 0.4 L - 0.45 L^2)y_t = (1 + L + 0.25 L^2) \epsilon_t.$$

Let's simulate this ARMA(2, 2) process

In [ ]:
set.seed(389745847)
phi1 <- .4
phi2 <- .45
theta1 <- 1
theta2 <- .25
T <- 1000
O <- 2
eps <- rnorm(T + O)
arma22 <- vector()
arma22[1] <- 0
arma22[2] <- 0
for (t in (O + 1): (T + O)){
    arma22[t] <- phi1 * arma22[t - 1] + phi2 * arma22[t - 2] + eps[t] + theta1 * eps[t - 1] + theta2 * eps[t - 2]
}

In [ ]:
plot.ts(arma22, main = 'ARMA(2, 2)', ylab = NA, xlab = NA)
par(mfrow = c(1, 2))
acf(arma22)
pacf(arma22)

The associated polynomials have a common factor, we can write
$$(1 - 0.4 L - 0.45 L^2)y_t = (1 + 0.5L)(1 - 0.9L)y_t,$$
and
$$(1 + L + 0.25 L^2) \epsilon_t = (1 + 0.5L) (1 + 0.5L) \epsilon_t.$$
The common factor $(1 + 0.5L)$ can be cancelled, hence we have
$$(1 - 0.9L)y_t = (1 + 0.5L) \epsilon_t,$$
which is an ARMA(1, 1). Let us simulate that too.

In [ ]:
set.seed(389745847)
phi1 <- .9
theta1 <- .5
T <- 1000
O <- 2
eps <- rnorm(T + O)
arma11 <- vector()
arma11[1] <- 0
arma11[2] <- 0
for (t in (O + 1): (T + O)){
    arma11[t] <- phi1 * arma11[t - 1] + eps[t] + theta1 * eps[t - 1]
}

In [ ]:
par(mfrow = c(1,2))
plot.ts(arma11, main = 'ARMA(1, 1)', ylab = NA, xlab = NA)
plot.ts(arma22, main = 'ARMA(2, 2)', ylab = NA, xlab = NA)

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 12)
par(mfrow = c(2, 2))
acf(arma11, main = 'ARMA(1, 1) - ACF')
acf(arma22, main = 'ARMA(2, 2) - ACF')
pacf(arma11, main = 'ARMA(1, 1) - PACF')
pacf(arma22, main = 'ARMA(2, 2) - PACF')

In [ ]:
plot(arma11 -  arma22)